In [ ]:
# Method 1: PCA + SVM
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC



# Load dataset
measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]

# Train-test split
# First, split into train (60%) and temp (40%) sets
trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')
print(f"Evaluation Data: {evalCIR.shape}")

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)
n_comp = 6


# -------------------------------------------------- Preprocessing ------------------------------------------
def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)  
    print(f"reshaped_data: {reshaped_data.shape}")
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    print(f"data_scaled: {data_scaled.shape}")
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

# -------- Train --------
# Feature Extraction
train_alice_cirs = trainCIR[:, alice_channel, :, :]
train_eve_cirs = trainCIR[:, eve_channel, :, :]

train_alice_mag = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_mag = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])

train_cirs = np.vstack((train_alice_mag, train_eve_mag)) # (cir, 251)
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp) # (cir, 2)
print(f'train_cirs_pca: {train_cirs_pca.shape}')

# -------- Test --------
# Feature Extraction
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_eve_cirs = testCIR[:, eve_channel, :, :]


test_alice_mag = np.abs(test_alice_cirs[..., 0] + 1j * test_alice_cirs[..., 1])
test_eve_mag = np.abs(test_eve_cirs[..., 0] + 1j * test_eve_cirs[..., 1]) # (cir, 251)
test_cirs = np.vstack((test_alice_mag, test_eve_mag)) # (cir, 251)
print(f'test_cirs: {test_cirs.shape}')

# Scale
test_cirs_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {test_cirs_scaled.shape}')
# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)
print(f'test_cirs_pca: {test_cirs_pca.shape}')

# ----------- Labels -----------
# train
train_alice_labels = np.zeros(train_alice_cirs.shape[0])
train_eve_labels = np.ones(train_eve_cirs.shape[0])
train_labels = np.hstack((train_alice_labels, train_eve_labels)) # (cir,) -> (1600,)
# test
test_alice_label = np.zeros(test_alice_cirs.shape[0])
test_eve_labels = np.ones(test_eve_cirs.shape[0])
test_labels = np.hstack((test_alice_label, test_eve_labels))



# ----------------- Classification -----------------

# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_cirs_pca, train_labels)

# Predict on test data
predictions = classifier.predict(test_cirs_pca)

# ----------------- Testing Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# -------------------------------------------------------- Evaluation --------------------------------------------------------
# Feature Extraction
eval_alice_cirs = evalCIR[:, alice_channel, :, :]
eval_eve_cirs = evalCIR[:, eve_channel, :, :]

eval_alice_mag = np.abs(eval_alice_cirs[..., 0] + 1j * eval_alice_cirs[..., 1])
eval_eve_mag = np.abs(eval_eve_cirs[..., 0] + 1j * eval_eve_cirs[..., 1]) # (cir, 251)
eval_cirs = np.vstack((eval_alice_mag, eval_eve_mag)) # (cir, 251)
# print(f'eval_cirs: {eval_cirs.shape}')

# Scale evaluation data
eval_cirs_scaled = scaler.transform(eval_cirs)
# print(f'eval_cirs_scaled: {eval_cirs_scaled.shape}')

# Apply PCA to evaluation data
eval_cirs_pca = pca.transform(eval_cirs_scaled)
# print(f'eval_cirs_pca: {eval_cirs_pca.shape}')

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_cirs.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_cirs.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))


# Predict on evaluation data
eval_predictions = classifier.predict(eval_cirs_pca)

# Calculate accuracy
eval_accuracy  = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")




trainData - (600, 15, 251, 2)
testData - (200, 15, 251, 2)
Evaluation Data: (200, 15, 251, 2)
reshaped_data: (1200, 251)
data_scaled: (1200, 251)
train_cirs_pca: (1200, 6)
test_cirs: (400, 251)
test_cirs_scaled: (400, 251)
test_cirs_pca: (400, 6)
Classification Accuracy: 76.50%
tp: 152
tn: 154
fp: 46
fn: 48
MDR: 0.23
FAR: 0.24
AR: 0.765


In [ ]:
# Method 2

import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import DictionaryLearning

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_varspeed_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_reflector_1.npz', allow_pickle=False)
measurement = np.load('../test/dataset/meas_asymm_reflector_2.npz', allow_pickle=False)

no_nonzero_coefs = 50
# no_nonzero_coefs = 100
# no_nonzero_coefs = 150
# no_nonzero_coefs = 200

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]
# First, split into train (60%) and temp (40%) sets
trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f"Train Data: {trainCIR.shape}")
print(f"Test Data: {testCIR.shape}")
print(f"Evaluation Data: {evalCIR.shape}")


# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(train_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(train_eve_magnitude.shape[0])       # Label '1' for Eve.

# Combine data and labels for training
train_atoms = train_data_combined
train_labels = np.hstack((alice_train_labels, eve_train_labels))
print(train_atoms.shape)
# Step 4: Form the Dictionary D from Training Data
D = train_atoms.T

# # Step 3: Extract Features for Test Data
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(test_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(test_eve_magnitude.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = np.vstack((test_alice_magnitude, test_eve_magnitude))
test_labels = np.hstack((alice_test_labels, eve_test_labels))


# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=no_nonzero_coefs):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    # print(reconstructed_signal)
    # print(tSample.shape)
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)


predictions = np.array(predictions)
# print(predictions.shape)


accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")


# -------- Evaluation --------
# Extract CIRs
eval_alice_CIRs = evalCIR[:, alice_channel, :, :]
eval_eve_CIRs = evalCIR[:, eve_channel, :, :]

# Compute magnitudes
eval_alice_magnitude = np.abs(eval_alice_CIRs[..., 0] + 1j * eval_alice_CIRs[..., 1])
eval_eve_magnitude = np.abs(eval_eve_CIRs[..., 0] + 1j * eval_eve_CIRs[..., 1])

# Stack evaluation features
eval_atoms = np.vstack((eval_alice_magnitude, eval_eve_magnitude))

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_magnitude.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_magnitude.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Step 8: Classifying Evaluation Data
eval_predictions = []

for evalSample in eval_atoms:
    predicted_class = classify_signal(evalSample, D, train_labels)
    eval_predictions.append(predicted_class)

eval_predictions = np.array(eval_predictions)

# Calculate accuracy
eval_accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")



Train Data: (600, 15, 251, 2)
Test Data: (200, 15, 251, 2)
Evaluation Data: (200, 15, 251, 2)
(1200, 251)
Classification Accuracy: 64.00%


In [ ]:
# Method 3 
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import DictionaryLearning

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]
# First, split into train (60%) and temp (40%) sets
trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f"Train Data: {trainCIR.shape}")
print(f"Test Data: {testCIR.shape}")
print(f"Evaluation Data: {evalCIR.shape}")

num = 5
n_comp = num
no_nonzero_coefs = num


# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# -------------------------------------------------- Preprocessing ------------------------------------------
def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)  
    print(f"reshaped_data: {reshaped_data.shape}")
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    print(f"data_scaled: {data_scaled.shape}")
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

train_cirs_pca, scaler, pca = apply_pca(train_data_combined, n_components=n_comp) # (cir, 2)
print(f"train_cirs_pca: {train_cirs_pca.shape}")

# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(train_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(train_eve_magnitude.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels))

# Combine data and labels for training
train_atoms = train_cirs_pca
print(train_atoms.shape)
# Step 4: Form the Dictionary D from Training Data
D = train_atoms.T


# # Step 3: Extract Features for Test Data
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

# Scale
test_cirs_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {test_cirs_scaled.shape}')
# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)
print(f'test_cirs_pca: {test_cirs_pca.shape}')

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(test_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(test_eve_magnitude.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Combine data and labels for testing
test_atoms = test_cirs_pca


# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=no_nonzero_coefs):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    # print(D.shape)
    # print(tSample.shape)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    # print(reconstructed_signal)
    # print(tSample.shape)
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)


predictions = np.array(predictions)
# print(predictions.shape)


accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# -------- Evaluation --------
# Extract CIRs
eval_alice_CIRs = evalCIR[:, alice_channel, :, :]
eval_eve_CIRs = evalCIR[:, eve_channel, :, :]

# Compute magnitudes
eval_alice_magnitude = np.abs(eval_alice_CIRs[..., 0] + 1j * eval_alice_CIRs[..., 1])
eval_eve_magnitude = np.abs(eval_eve_CIRs[..., 0] + 1j * eval_eve_CIRs[..., 1])

# Stack evaluation features
eval_atoms = np.vstack((eval_alice_magnitude, eval_eve_magnitude))

# Scale
eval_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {eval_scaled.shape}')
# PCA
eval_cirs = pca.transform(eval_scaled)
print(f'test_cirs_pca: {eval_cirs.shape}')

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_magnitude.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_magnitude.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Step 8: Classifying Evaluation Data
eval_predictions = []

for evalSample in eval_cirs:
    predicted_class = classify_signal(evalSample, D, train_labels)
    eval_predictions.append(predicted_class)

eval_predictions = np.array(eval_predictions)

# Calculate accuracy
eval_accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


In [ ]:
# Method 4 - Sparse DL - SVM
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../test/dataset/meas_asymm_reflector_2.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f"Train Data: {trainCIR.shape}")
print(f"Test Data: {testCIR.shape}")
print(f"Evaluation Data: {evalCIR.shape}")

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

n_components = 5 
# n_components = 10
# n_components = 15
# n_components = 20


# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs= -1)
dict_learner.fit(train_data_combined)

train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))


test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))


# Transform test data into sparse codes
test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# Combine the sparse codes
test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))


# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


scaler = StandardScaler()
train_sparse_codes = scaler.fit_transform(train_sparse_codes)
# Apply the same scaling to test data
test_sparse_codes = scaler.transform(test_sparse_codes)
print('Train data shape:', train_sparse_codes.shape)

# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_sparse_codes, train_labels)

# Predict on test data
predictions = classifier.predict(test_sparse_codes)


# -------- Evaluation --------
# Extract CIRs
eval_alice_CIRs = evalCIR[:, alice_channel, :, :]
eval_eve_CIRs = evalCIR[:, eve_channel, :, :]

# Compute magnitudes
eval_alice_magnitude = np.abs(eval_alice_CIRs[..., 0] + 1j * eval_alice_CIRs[..., 1])
eval_eve_magnitude = np.abs(eval_eve_CIRs[..., 0] + 1j * eval_eve_CIRs[..., 1])

# Stack evaluation features
eval_cirs = np.vstack((eval_alice_magnitude, eval_eve_magnitude))

# Transform evaluation data into sparse codes
eval_alice_sparse_codes = dict_learner.transform(eval_alice_magnitude)
eval_eve_sparse_codes = dict_learner.transform(eval_eve_magnitude)

# Combine the sparse representations
eval_sparse_codes = np.vstack((eval_alice_sparse_codes, eval_eve_sparse_codes))
# print('Evaluation data shape:', eval_sparse_codes.shape)

# Scale evaluation data using the trained scaler
eval_sparse_codes = scaler.transform(eval_sparse_codes)

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_sparse_codes.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_sparse_codes.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Predict on evaluation data
eval_predictions = classifier.predict(eval_sparse_codes)

eval_accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")




Train Data: (600, 15, 251, 2)
Test Data: (200, 15, 251, 2)
Evaluation Data: (200, 15, 251, 2)
[44563.836674    6678.21543716  1459.85407267 -3552.94847113
 -5416.00109494  1357.92339736  2812.1617227  13351.5434469
 -5547.6442559   1667.63009017  1251.14012031  -439.58181313
  4304.26649799   640.04911611 -1174.39239528  6188.34186048
  3176.84007738   484.98325108  -351.82529531 -3919.6115697 ]
Test data shape: (400, 20)
Train data shape: (1200, 20)


In [ ]:
# Method 2
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import DictionaryLearning

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)

# no_nonzero_coefs = 2

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]
# First, split into train (60%) and temp (40%) sets
trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

num = 5
n_comp = num
no_nonzero_coefs = num

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# -------------------------------------------------- Preprocessing ------------------------------------------
def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)  
    print(f"reshaped_data: {reshaped_data.shape}")
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    print(f"data_scaled: {data_scaled.shape}")
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

train_cirs_pca, scaler, pca = apply_pca(train_data_combined, n_components=n_comp) # (cir, 2)
print(f"train_cirs_pca: {train_cirs_pca.shape}")

# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(train_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(train_eve_magnitude.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels))

# Combine data and labels for training
train_atoms = train_cirs_pca
print(train_atoms.shape)
# Step 4: Form the Dictionary D from Training Data
D = train_atoms.T


# # Step 3: Extract Features for Test Data
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

# Scale
test_cirs_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {test_cirs_scaled.shape}')
# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)
print(f'test_cirs_pca: {test_cirs_pca.shape}')

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(test_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(test_eve_magnitude.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Combine data and labels for testing
test_atoms = test_cirs_pca


# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=no_nonzero_coefs):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    # print(D.shape)
    # print(tSample.shape)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    # print(reconstructed_signal)
    # print(tSample.shape)
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)


predictions = np.array(predictions)
# print(predictions.shape)


accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")


# -------- Evaluation --------
# Extract CIRs
eval_alice_CIRs = evalCIR[:, alice_channel, :, :]
eval_eve_CIRs = evalCIR[:, eve_channel, :, :]

# Compute magnitudes
eval_alice_magnitude = np.abs(eval_alice_CIRs[..., 0] + 1j * eval_alice_CIRs[..., 1])
eval_eve_magnitude = np.abs(eval_eve_CIRs[..., 0] + 1j * eval_eve_CIRs[..., 1])

# Stack evaluation features
eval_atoms = np.vstack((eval_alice_magnitude, eval_eve_magnitude))

# Scale
eval_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {eval_scaled.shape}')
# PCA
eval_cirs = pca.transform(eval_scaled)
print(f'test_cirs_pca: {eval_cirs.shape}')

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_magnitude.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_magnitude.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Step 8: Classifying Evaluation Data
eval_predictions = []

for evalSample in eval_cirs:
    predicted_class = classify_signal(evalSample, D, train_labels)
    eval_predictions.append(predicted_class)

eval_predictions = np.array(eval_predictions)

# Calculate accuracy
eval_accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

